In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split

In [3]:
df=pd.read_excel('/content/drive/MyDrive/NNFL-Assignment2/data5.xlsx',header=None)
df.head()

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [4]:
df = df.drop_duplicates(keep='first')
df = df.sample(frac = 1, random_state = 7).reset_index(drop=True)
m = df.shape[0]
n = df.shape[1] - 1
print(m,n)
df.head()

210 7


,0,1,2,3,4,5,6,7
0,18.94,16.49,0.8750,6.445,3.639,5.0640,6.362,2
1,12.88,13.50,0.8879,5.139,3.119,2.3520,4.607,1
2,18.36,16.52,0.8452,6.666,3.485,4.9330,6.448,2
3,15.88,14.90,0.8988,5.618,3.507,0.7651,5.091,1
4,13.22,13.84,0.8680,5.395,3.070,4.1570,5.088,1


In [5]:
datan = df.values
X = datan[:, 0:7]

X = (X - np.min(X, axis = 0))/(np.max(X, axis=0)-np.min(X, axis=0))
X = np.concatenate((np.ones((m,1)),X), axis=1)
#X = (X - X.mean(axis = 0))/(X.std(axis = 0))
y = datan[:, -1:]
print(X.shape, y.shape)

(210, 8) (210, 1)


In [6]:
num_classes = len(np.unique(y))
print(num_classes)

3


In [7]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(151, 8) (17, 8) (42, 8) (151, 1) (17, 1) (42, 1)


In [8]:
def one_hot(y):
  len = np.size(y)
  encoded_y = np.zeros((len,3))
  for i in range(len):
    if y[i] == 1:
      encoded_y[i,0] = 1
    if y[i] == 2:
      encoded_y[i,1] = 1
    if y[i] == 3:
      encoded_y[i,2] = 1
  return encoded_y

In [9]:
y_train_encoded = one_hot(y_train)
y_val_encoded = one_hot(y_val)
y_test_encoded = one_hot(y_test)

In [10]:
x_d= datan[:,0:7]
y_d= datan[:,7:8]

x_norm=np.ones((x_d.shape[0],x_d.shape[1]+1))
for i in range(x_d.shape[1]):
  x_norm[:,i+1:i+2]=(x_d[:,i:i+1]-np.amin(x_d[:,i:i+1]))/(np.amax(x_d[:,i:i+1])-np.amin(x_d[:,i:i+1]))

In [11]:
y_norm=np.zeros((len(y_d),3))

for i in range(len(y_d)):
  if y_d[i][0]==1:
    y_norm[i][0]=1
    y_norm[i][1]=0
    y_norm[i][2]=0
  elif y_d[i][0]==2:
    y_norm[i][0]=0
    y_norm[i][1]=1
    y_norm[i][2]=0
  elif y_d[i][0]==3:
    y_norm[i][0]=0
    y_norm[i][1]=0
    y_norm[i][2]=1

In [12]:
def divide_tr_va_te(x_norm,y_norm):
  x_tr=x_norm[:math.floor(0.7*x_norm.shape[0])]
  x_va=x_norm[math.floor(0.7*x_norm.shape[0]):math.floor(0.8*x_norm.shape[0])]
  x_te=x_norm[math.floor(0.8*x_norm.shape[0]):]

  y_tr=y_norm[:math.floor(0.7*y_norm.shape[0])]
  y_va=y_norm[math.floor(0.7*y_norm.shape[0]):math.floor(0.8*y_norm.shape[0])]
  y_te=y_norm[math.floor(0.8*y_norm.shape[0]):]
  return x_tr,x_va,x_te,y_tr,y_va,y_te

x_tr,x_va,x_te,y_tr,y_va,y_te=divide_tr_va_te(x_norm,y_norm)

In [13]:
def encode(x,w):
  return x@w.T

In [14]:
def sigmoid(z):
  return 1/(1+np.exp(-z))

In [15]:
def elm(x,y,hidden):
  random_mat=np.random.randn(x.shape[1],hidden)
  h=x@random_mat
  h=sigmoid(h)

  w=np.linalg.pinv(h)@y
  return random_mat,w

In [16]:
def hot_decoding(mat):
  dec=np.zeros((len(mat),1))
  for i in range(len(mat)):
    dec[i]=np.argmax(mat[i])+1
  return dec

In [17]:
def cost(predictions,labels):
  return np.mean(np.mean((predictions-labels)**2,axis=1))

In [18]:
def autoencoder(x_tr,x_va,min_hidden,max_hidden):
  hidden_arr=np.arange(min_hidden,max_hidden,50)
  hidden_max=0
  err_min=float('inf')
  random_mat_max=0
  w_max=0

  y_te_dec=hot_decoding(y_te)

  for i in range(len(hidden_arr)):
      hidden=hidden_arr[i]
      random_mat,w=elm(x_tr,x_tr,hidden)

      h=x_va@random_mat
      h=sigmoid(h)
      pred_va=h@w

      err=np.mean(np.mean((pred_va-x_va)**2,axis=1))
      if err<=err_min:
          err_min=err
          hidden_max=hidden
          random_mat_max=random_mat
          w_max=w
  return w_max,err_min,hidden_max

w1,err1,hidden1=autoencoder(x_tr,x_va,50,2000)

In [19]:
h_tr=encode(x_tr,w1)
h_va=encode(x_va,w1)

w2,err2,hidden2=autoencoder(h_tr,h_va,50,hidden1)

h2_tr=encode(h_tr,w2)
h2_va=encode(h_va,w2)

min_hidden=50
max_hidden=2000
hidden_arr=np.arange(min_hidden,max_hidden,50)
hidden_max=0
err_min=float('inf')
random_mat_max=0
w_max=0

for i in range(len(hidden_arr)):
      hidden=hidden_arr[i]
      random_mat,w=elm(h2_tr,y_tr,hidden)

      h=h2_va@random_mat
      h=sigmoid(h)
      pred_va=h@w
      err=np.mean(np.mean((pred_va-y_va)**2,axis=1))
      if err<=err_min:
          err_min=err
          hidden_max=hidden
          random_mat_max=random_mat
          w_max=w

In [20]:
h1_te=encode(x_te,w1)
h2_te=encode(h1_te,w2)

h=h2_te@random_mat_max
h=sigmoid(h)
pred=h@w_max

pred_dec=hot_decoding(pred)
y_te_dec=hot_decoding(y_te)

In [21]:
def confusion_matrix(predictions, labels):
  conf_matrix = [0,0,0,0,0,0,0,0,0]
  m = labels.shape[0]
  for i in range(m):
    conf_matrix[3*(labels[i]-1) + predictions[i] - 1] = conf_matrix[3*(labels[i]-1) + predictions[i] - 1] + 1 
  return conf_matrix

In [22]:
conf_matrix = confusion_matrix(pred_dec.flatten().astype(int), y_te_dec.flatten().astype(int))

ind_acc1=conf_matrix[0]/sum(conf_matrix[0:3])
ind_acc2=conf_matrix[4]/sum(conf_matrix[3:6])
ind_acc3=conf_matrix[8]/sum(conf_matrix[6:9]) 
overall=(conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix)

In [23]:
print("Individual accuracy 1: ",ind_acc1)
print("Individual accuracy 2: ",ind_acc2)
print("Individual accuracy 3: ",ind_acc3)
print("Overall accuracy : ", overall)

Individual accuracy 1:  0.875
Individual accuracy 2:  1.0
Individual accuracy 3:  1.0
Overall accuracy :  0.9523809523809523
